In [1]:
import sys
sys.path.append('../')

import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from util import load_jsonl, load_node_csv

from model.continuous_prompt import ContinuousPromptingLLM
from model.rec_encoder import RecommendationContinuousPromptModel
from model.projection import BasicProjection

In [2]:
data_path='/home/bonbak/kobaco/data/kobaco.csv'
device='cuda:3'
save_dir = "/SSL_NAS/bonbak/model/models--yanolja--EEVE-Korean-Instruct-2.8B-v1.0/snapshots/482db2d0ba911253d09342c34d0e42ac871bfea3"
tokenizer = AutoTokenizer.from_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(save_dir).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_path):
        self.data = load_jsonl(data_path)
        self.input_text_list = self.set_input_text_list()
        self.answer_list = self.set_answer_list()
        self.continuous_prompt_input_list = self.set_continuous_prompt_input_list()

    def set_input_text_list(self):
        input_text_list = []
        for data in self.data:
            user = data['user_id']
            prompt =f'사용자 {user}의 TV 프로그램 시청 기록:\n'
            for idx, item in enumerate(data['iteracted_items']):
                prompt += f'{idx}. {item}\n'
            prompt +='\n타겟 TV 프로그램:\n* ' + data['target_item'] + '\n\n'
            prompt += data['question']
            input_text_list.append(prompt)
        return input_text_list

    def set_answer_list(self):
        return [x['answer'] for x in self.data]
    

    def set_continuous_prompt_input_list(self):
        # return [{'input_text_list':'\n'.join([x['node_information'],x['edge_information']])} for x in self.data]
        return

    def __len__(self):
        return len(self.input_text_list)

    def __getitem__(self, idx):
        return self.input_text_list[idx], self.answer_list[idx]

In [8]:
user_mapping = load_node_csv('../data/kobaco.csv', index_col='user_id')
item_mapping = load_node_csv('../data/kobaco.csv', index_col='item_id')

class RecommendationDataset(Dataset):
    def __init__(self, data_path, user_mapping, item_mapping):
        self.user_mapping = user_mapping
        self.item_mapping = item_mapping
        super().__init__(data_path)
        

    def set_continuous_prompt_input_list(self):
        continuous_prompt_input_list = []
        for x in self.data:
            interacted_items = list(map(lambda item:self.item_mapping[item], x['iteracted_items']))
            target_item = [self.item_mapping[x['target_item']]]
            item_ids = torch.Tensor(interacted_items+target_item).type(torch.long)
            # user_id = torch.Tensor([self.user_mapping[x['user_id']]]).type(torch.long)
            user_id = self.user_mapping[x['user_id']]
            continuous_prompt_input_list.append({'user_id':user_id, 'item_ids':item_ids})
        return continuous_prompt_input_list

    def __getitem__(self, idx):
        return self.input_text_list[idx], self.continuous_prompt_input_list[idx], self.answer_list[idx]

In [9]:
SAVE_DIR = '../output'
MODEL_NAME = 'light-gcn'

test_dataset = RecommendationDataset('../data/test.jsonl', user_mapping, item_mapping)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

num_users, num_items = len(user_mapping), len(item_mapping)

continuous_prompt_model = RecommendationContinuousPromptModel(num_users,num_items,'../data/train_edge_index.pt')
projection_module = BasicProjection(continuous_prompt_model.model.embedding_dim)

In [10]:
model = ContinuousPromptingLLM(
    "/SSL_NAS/bonbak/model/models--yanolja--EEVE-Korean-Instruct-2.8B-v1.0/snapshots/482db2d0ba911253d09342c34d0e42ac871bfea3",
    continuous_prompt_model, 
    continuous_prompt_model.model.embedding_dim
)

model.continuous_prompt_model.load_state_dict(torch.load(f'{SAVE_DIR}/model/{MODEL_NAME}-encoder.bin'))
model.projection_module.load_state_dict(torch.load(f'{SAVE_DIR}/model/{MODEL_NAME}-projection.bin'))

continuous_prompt_model.to(device)
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ContinuousPromptingLLM(
  (llm_model): PhiForCausalLM(
    (model): PhiModel(
      (embed_tokens): Embedding(58944, 2560)
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-31): 32 x PhiDecoderLayer(
          (self_attn): PhiSdpaAttention(
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (dense): Linear(in_features=2560, out_features=2560, bias=True)
            (rotary_emb): PhiRotaryEmbedding()
          )
          (mlp): PhiMLP(
            (activation_fn): NewGELUActivation()
            (fc1): Linear(in_features=2560, out_features=10240, bias=True)
            (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          )
          (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (resid_dropout): Drop

In [19]:
from tqdm.notebook import tqdm
model.eval()
pred = []
label = []

for input_text, continuous_prompt_input, answer_list in tqdm(test_dataloader):
    with torch.no_grad():
        inputs_embeds, attention_mask = model.make_input_embed(input_text, continuous_prompt_input, embedding_first=True)
        output = model.llm_model.generate(inputs_embeds=inputs_embeds, attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1)
        pred.append(model.llm_tokenizer.batch_decode(output, skip_special_tokens=True)[0])
        label.append(answer_list)

  0%|          | 0/5666 [00:00<?, ?it/s]

In [21]:
import numpy as np
def convert_answer(answer):
    converted = []
    for a in answer:
        a = a.strip()
        if a == '예':
            converted.append(1)
        elif a == '아니':
            converted.append(0)
        else:
            converted.append(-1)
    return np.array(converted)

In [22]:
y_pred = convert_answer(pred)
y_true = convert_answer(test_dataset.answer_list)
new_y_pred = y_pred[np.where(y_pred!=-1)]
new_y_true = y_true[np.where(y_pred!=-1)]
missed = np.where(y_pred==-1)
print('miss rate:',len(missed[0]) / len(y_true))

miss rate: 0.0


In [23]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(new_y_true, new_y_pred)
f1 = f1_score(new_y_true, new_y_pred)

print(accuracy)
print(f1)

0.8210377691493117
0.8098987626546682
